# Amazon Bedrock AgentCore Runtime での分散マルチエージェントソリューション

## 概要

このチュートリアルでは、異なる Agentic フレームワークで構築された複数のエージェントを、それぞれ独自の Bedrock AgentCore Runtime に独立してホストする方法を学びます。その後、分散マルチエージェントソリューションとしてそれらの間の通信を有効にします。

この例では以下を作成します：
1. プログラミングや技術トラブルシューティングに関する技術的な質問に回答することを専門とするテクニカルエージェント（`tech_agent`）
2. 会社の福利厚生を専門とする HR エージェント（`hr_agent`）
3. 質問をテクニカルエージェントまたは HR エージェントにルーティングするオーケストレーターエージェント（`orchestrator_agent`）

これら3つのエージェントを組み合わせることで、スーパーバイザーを持つマルチエージェント構成が得られ、ユーザーの質問を適切なサブエージェントにルーティングできます。このシステムは、従業員が会社で持つ可能性のある様々な質問に回答できます。


### チュートリアル詳細


| 情報                | 詳細                                                                             |
|:--------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ | 対話型                                                                           |
| エージェントタイプ   | マルチエージェント（スーパーバイザーがエージェントをツールとして呼び出す）       |
| エージェントフレームワーク | Strands Agents & LangGraph                                                 |
| LLM モデル          | Anthropic Claude Haiku 4.5                                                       |
| チュートリアルコンポーネント | AgentCore Runtime でのエージェントホスティングとマルチエージェント連携の有効化 |
| チュートリアル分野   | クロスバーティカル                                                               |
| 難易度              | 中級                                                                             |
| 使用 SDK            | Amazon BedrockAgentCore Python SDK および boto3                                  |

### チュートリアルアーキテクチャ

このチュートリアルでは、3つのエージェントを Bedrock AgentCore Runtime にデプロイする方法を説明します。オーケストレーターには Strands Agent、テクニカルエージェントには Strands Agent、HR エージェントには LangGraph エージェントを使用します。シンプルなエージェントを使用して、各エージェントが独自の AgentCore Runtime にデプロイされた、異なるエージェントフレームワークを組み合わせたマルチエージェントシステムの構成方法をデモンストレーションします。

![alt text](./architecture.png)


### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime での複数エージェントのホスティング
* 各エージェントが独立してホストされるマルチエージェントソリューションの作成

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS 資格情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* LangGraph

In [ ]:
!uv add -r requirements.txt --active

In [ ]:
import os

# 環境変数を設定
os.environ["AWS_DEFAULT_REGION"] = "us-west-2"

## エージェントの作成

まず、各エージェント用に3つの個別の IAM ロールを作成します。これにより、各エージェントに対して他のエージェントとは独立して最小権限のパーミッションを定義できます。

In [ ]:
from utils import create_agentcore_role

tech_agent_name="tech_agent"
tech_agent_iam_role = create_agentcore_role(agent_name=tech_agent_name, region=os.getenv("AWS_DEFAULT_REGION"))
tech_agent_role_arn = tech_agent_iam_role['Role']['Arn']
tech_agent_role_name = tech_agent_iam_role['Role']['RoleName']
print(f"テクニカルエージェント ロールARN: {tech_agent_role_arn}")
print(f"テクニカルエージェント ロール名: {tech_agent_role_name}")

hr_agent_name="hr_agent"
hr_agent_iam_role = create_agentcore_role(agent_name=hr_agent_name, region=os.getenv("AWS_DEFAULT_REGION"))
hr_agent_role_arn = hr_agent_iam_role['Role']['Arn']
hr_agent_role_name = hr_agent_iam_role['Role']['RoleName']
print(f"HRエージェント ロールARN: {hr_agent_role_arn}")
print(f"HRエージェント ロール名: {hr_agent_role_name}")

orchestrator_agent_name="orchestrator_agent"
orchestrator_iam_role = create_agentcore_role(agent_name=orchestrator_agent_name, region=os.getenv("AWS_DEFAULT_REGION"))
orchestrator_role_arn = orchestrator_iam_role['Role']['Arn']
orchestrator_role_name = orchestrator_iam_role['Role']['RoleName']
print(f"オーケストレーター ロールARN: {orchestrator_role_arn}")
print(f"オーケストレーター ロール名: {orchestrator_role_name}")

### ヘルパー関数：

* `configure_runtime` ヘルパー関数は、各エージェントのランタイム設定をセットアップするために使用されます。この例では、スターターツールキットを使用して、エントリポイント、作成した実行ロール、requirements ファイルで AgentCore Runtime デプロイメントを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。
* `check_status` ヘルパー関数は、AWS アカウントにデプロイされた各ランタイムをチェックして、作成が成功し、エージェントが使用準備ができていることを検証するために使用されます。

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import time


def configure_runtime(agent_name, agentcore_iam_role, python_file_name):
    boto_session = Session(region_name=os.getenv("AWS_DEFAULT_REGION"))
    region = boto_session.region_name

    agentcore_runtime = Runtime()

    response = agentcore_runtime.configure(
        entrypoint=python_file_name,
        execution_role=agentcore_iam_role['Role']['Arn'],
        auto_create_ecr=True,
        requirements_file="requirements.txt",
        region=region,
        agent_name=agent_name
    )
    return response, agentcore_runtime

def check_status(agent_runtime):
    status_response = agent_runtime.status()
    status = status_response.endpoint['status']
    end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
    while status not in end_status:
        time.sleep(10)
        status_response = agent_runtime.status()
        status = status_response.endpoint['status']
        print(status)
    return status

In [ ]:
# 現在の作業ディレクトリを tech_agent フォルダに設定
import os
os.chdir('./tech_agent')
print(f"現在のディレクトリ: {os.getcwd()}")

### テクニカルサポートエージェントの作成（Strands Agents）

まず、Strands と Amazon Bedrock モデルを使用したテクニカルサポートエージェントから始めましょう。次のセルを実行すると、`./tech_agent` ディレクトリにエージェント固有のロジックを持つ `tech_agent.py` ファイルが作成されます。

アプリは `BedrockAgentCoreApp()` で定義され、呼び出し関数 `strands_agent_bedrock` は `@app.entrypoint` デコレーターで装飾され、ファイルの最後に `app.run()` コマンドがあることに注意してください。

In [ ]:
%%writefile tech_agent.py

from strands import Agent, tool
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel

app = BedrockAgentCoreApp()

model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    system_prompt="You're a helpful tech support assistant, you can help user questions on tech troubleshooting and programming"
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    user_input = payload.get("prompt")
    print("ユーザー入力:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

#### エージェントの起動：

まず、configure_runtime ヘルパー関数を使用して、エージェントデプロイメントに必要な .bedrock_agentcore.yaml、.dockerignore、Dockerfile を作成します。その後、ランタイムで .launch() を呼び出して、イメージを ECR にプッシュし、AWS 環境に AgentCore Runtime を作成します。

In [ ]:
_, tech_agent_runtime = configure_runtime("tech_agent", tech_agent_iam_role, "tech_agent.py")
tech_launch_result = tech_agent_runtime.launch()
tech_agent_id = tech_launch_result.agent_id
tech_agent_arn = tech_launch_result.agent_arn

print(f"テクニカルエージェント ARN: {tech_agent_arn}")

#### テクニカルエージェント ARN をパラメータストアに保存

テクニカルエージェントの AgentCore Runtime ARN を永続的に保存し、検索できるようにシンプルなエージェントレジストリを作成します。

In [ ]:
import boto3
import time

ssm = boto3.client('ssm')
ssm.put_parameter(
    Name=f'/agents/tech_agent_arn',
    Value=tech_agent_arn,
    Type='String',
    Overwrite=True
)

#### エージェントのテスト

エージェントをテストするために、まずテクニカルエージェント AgentCore Runtime のステータスを確認し、使用準備ができていることを確認しましょう。
tech_agent_runtime で `.invoke()` を使用して、エージェントランタイムが正しく設定され、期待通りに動作していることを検証します。

In [ ]:
status = check_status(tech_agent_runtime)
print(f"ステータス: {status}")

In [ ]:
invoke_response = tech_agent_runtime.invoke({"prompt": "shortcut to minimize windows in Mac, in 1 sentence"})
invoke_response

### HR エージェントの作成（LangGraph Agents）

同様のプロセスで HR エージェントを作成します。ただし、今回は基盤となるエージェントロジックが LangGraph で構築されていることに気付くでしょう。エージェントフレームワークの変更は、AgentCore Runtime の設定方法に影響を与えません。次のセルを実行すると、`./hr_agent` ディレクトリに `hr_agent.py` ファイルが作成されます。

テクニカルサポートエージェントと同様に、アプリは `BedrockAgentCoreApp()` で定義され、呼び出し関数 langgraph_bedrock は @app.entrypoint デコレーターで装飾され、ファイルの最後に `app.run()` コマンドがあります。

In [ ]:
# 現在の作業ディレクトリを hr_agent フォルダに設定
import os

os.chdir('../hr_agent')
print(f"現在のディレクトリ: {os.getcwd()}")

In [ ]:
%%writefile hr_agent.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
import operator
import math

app = BedrockAgentCoreApp()

@tool
def get_vacation_info():
    """今年の残り有給休暇日数を取得"""  # ダミー実装
    return "you have 12 days off remaining this year"

# 手動 LangGraph 構築を使用してエージェントを定義
def create_agent():
    """LangGraph エージェントを作成して設定"""
    from langchain_aws import ChatBedrock
    
    # LLM を初期化（必要に応じてモデルとパラメータを調整）
    llm = ChatBedrock(
        model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",  # または希望のモデル
        model_kwargs={"temperature": 0.1}
    )
    
    # ツールを LLM にバインド
    tools = [get_vacation_info]
    llm_with_tools = llm.bind_tools(tools)
    
    # システムメッセージ
    system_message = f"""You're a helpful hr support assistant, you can answers user questions on vacations and benefits. 
    Here are the primary company benefits
    - Comprehensive health insurance with 100% premium coverage for employees and 75% for dependents
    - Flexible PTO policy with 20 days paid vacation annually, plus 5 sick days
    - 401(k) plan with 6% company matching and immediate vesting
    - Monthly wellness stipend of $100 for gym memberships or fitness activities

    For additional HR information instruct the user to call to 1-800-ASKHR"""
    
    # chatbot ノードを定義
    def chatbot(state: MessagesState):
        # システムメッセージがまだ存在しない場合は追加
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # グラフを作成
    graph_builder = StateGraph(MessagesState)
    
    # ノードを追加
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # エッジを追加
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # エントリポイントを設定
    graph_builder.set_entry_point("chatbot")
    
    # グラフをコンパイル
    return graph_builder.compile()

# エージェントを初期化
agent = create_agent()

@app.entrypoint
def langgraph_bedrock(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    user_input = payload.get("prompt")
    
    # LangGraph が期待する形式で入力を作成
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # 最終メッセージの内容を抽出
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()

#### エージェントの起動：

同様に、configure_runtime ヘルパー関数を使用して、エージェントデプロイメントに必要な .bedrock_agentcore.yaml、.dockerignore、Dockerfile を作成します。その後、HR エージェントランタイムで .launch() を呼び出して、イメージを ECR にプッシュし、AWS 環境に AgentCore Runtime を作成します。

In [ ]:
_, hr_agentcore_runtime = configure_runtime("hr_agent", hr_agent_iam_role, "hr_agent.py")
hr_launch_result = hr_agentcore_runtime.launch()
hr_agent_id = hr_launch_result.agent_id
hr_agent_arn = hr_launch_result.agent_arn

print(f"HRエージェント ARN: {hr_agent_arn}")

#### HR エージェント ARN をパラメータストアに保存

HR エージェントの AgentCore Runtime ARN を永続的に保存し、検索できるようにシンプルなエージェントレジストリを継続します。

In [ ]:
import boto3
import time

ssm = boto3.client('ssm')
ssm.put_parameter(
    Name=f'/agents/hr_agent_arn',
    Value=hr_agent_arn,
    Type='String',
    Overwrite=True  
)

#### エージェントのテスト

HR エージェント AgentCore Runtime のステータスを確認し、使用準備ができていることを確認しましょう。
hr_agentcore_runtime で `.invoke()` を使用して、AgentCore Runtime が正しく設定され、期待通りに動作していることを検証します。

In [ ]:
status = check_status(hr_agentcore_runtime)
status

In [ ]:
# エージェントをテスト
invoke_response = hr_agentcore_runtime.invoke({"prompt": "How many vacation days I have left?"})
invoke_response

### オーケストレーターエージェントの作成（Strands Agents）

3番目のエージェント、オーケストレーターには、再び Strands をエージェントフレームワークとして使用します。エージェントを作成する前に、先ほど作成した AgentCore Runtime の実行ロールを更新して、テクニカルサポートエージェントと HR エージェントを呼び出す権限を許可する必要があります。

以下の `update_orchestrator_permissions` 関数は、サブエージェントの ARN とパラメータストアに登録されたエージェントの ARN を受け取り、オーケストレーターエージェントにサブエージェントの DEFAULT ランタイムエンドポイントを呼び出す権限を与えます。また、オーケストレーターエージェントにパラメータストアからエージェント ARN を取得する権限も与えます。

In [ ]:
# オーケストレーターの agentcore 実行ロールを更新して、必要なサブエージェントを呼び出す権限を付与
# オーケストレーターはパラメータストアからサブエージェントの ARN を取得する権限も必要
import json 

# パラメータストアからランタイム ARN を取得
ssm = boto3.client('ssm')
response = ssm.get_parameter(Name='/agents/tech_agent_arn')
tech_agent_arn = response['Parameter']['Value']
tech_agent_parameter_arn = response['Parameter']['ARN']

ssm = boto3.client('ssm')
response = ssm.get_parameter(Name='/agents/hr_agent_arn')
hr_agent_arn = response['Parameter']['Value']
hr_agent_parameter_arn = response['Parameter']['ARN']

def update_orchestrator_permissions(sub_agent_arns: list, sub_agent_parameter_arns: list, orchestrator_name: str):
    iam_client = boto3.client('iam')
    orchestrator_permissions = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "bedrock-agentcore:InvokeAgentRuntime"
                ],
                "Resource": [ sub_agent_arn + "/runtime-endpoint/DEFAULT" for sub_agent_arn in sub_agent_arns ] + [ sub_agent_arn for sub_agent_arn in sub_agent_arns ]
            },
            {
                "Effect": "Allow",
                "Action": [
                    "ssm:GetParameter"
                ],
                "Resource": [sub_agent_parameter_arn for sub_agent_parameter_arn in sub_agent_parameter_arns]

            }]
    }
        
    rsp = iam_client.put_role_policy(
        RoleName=orchestrator_name,
        PolicyName="subagent_permissions-new",
        PolicyDocument=json.dumps(orchestrator_permissions)
    )
    return rsp

rsp = update_orchestrator_permissions([tech_agent_arn, hr_agent_arn], [tech_agent_parameter_arn, hr_agent_parameter_arn], orchestrator_role_name)
print(f"権限の更新結果: {rsp}")

In [ ]:
# 現在の作業ディレクトリを orchestrator_agent フォルダに設定
import os
os.chdir('../orchestrator_agent')
print(f"現在のディレクトリ: {os.getcwd()}")

次のセルを実行すると、`./orchestrator_agent` ディレクトリにエージェント固有のロジックを持つ `orchestrator_agent.py` ファイルが作成されます。オーケストレーターエージェントには2つのツールが利用可能です：1. `call_tech_agent` と 2. `call_HR_agent`。これらのツールは両方とも、`./orchestrator_agent` フォルダに既にある `invoke_agent_utils.py` ファイルで事前定義された invoke_agent_utils 関数を使用します。サブエージェントは boto3 で利用可能な invoke_agent_runtime アクションを使用してツールとして呼び出されます。

このより複雑なエージェント設定でも、Bedrock AgentCore Runtime の設定は同じです。アプリは `BedrockAgentCoreApp()` で定義され、呼び出し関数 `strands_agent_bedrock_streaming` は `@app.entrypoint` デコレーターで装飾され、ファイルの最後に `app.run()` コマンドがあることに注意してください。

In [ ]:
%%writefile orchestrator_agent.py

import argparse
import json
import boto3
import logging

from strands import Agent, tool
from strands_tools import calculator 
from strands.models import BedrockModel

from bedrock_agentcore.runtime import BedrockAgentCoreApp

from invoke_agent_utils import invoke_agent_with_boto3

logger = logging.getLogger(__name__)

app = BedrockAgentCoreApp()

def get_agent_arn(agent_name: str) -> str:
    """
    Parameter Store からエージェント ARN を取得
    """
    try:
        ssm = boto3.client('ssm')
        response = ssm.get_parameter(
            Name=f'/agents/{agent_name}_arn'
        )
        return response['Parameter']['Value']
    except Exception as err:
        print(err)
        raise err

@tool
def call_tech_agent(user_query):
    """ テクニカルエージェントを呼び出す """ 
    # print("テクニカルエージェントを呼び出し中")
    try:
        tech_agent_arn = get_agent_arn ("tech_agent")
        result = invoke_agent_with_boto3(tech_agent_arn, user_query=user_query)
    except Exception as e:
        result = str(e)
        logger.exception("テクニカルエージェント呼び出し中の例外: ")
    return result

@tool
def call_HR_agent(user_query):
    """ HR エージェントを呼び出す """ 
    print("HR エージェントを呼び出し中")
    try:
        hr_agent_arn = get_agent_arn("hr_agent")
        print(hr_agent_arn)
        result = invoke_agent_with_boto3(hr_agent_arn, user_query=user_query)
    except Exception as e:
        result = str(e)
        logger.error(f"HR エージェント呼び出し中の例外: {e}", exc_info=True)
    return result


model_id = "global.anthropic.claude-haiku-4-5-20251001-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    system_prompt="You're a helpful assistant, your role is to understand user questions and delegate to the appropriate specialized agent, you have tools to call the tech and HR agents",
    tools=[call_tech_agent, call_HR_agent]
)

def parse_event(event):
    """
    エージェントからのストリーミングイベントを解析し、フォーマットされた出力を返す
    """
    # 表示不要なイベントをスキップ
    if any(key in event for key in ['init_event_loop', 'start', 'start_event_loop']):
        return ""
    
    # スーパーバイザーからのテキストチャンク
    if 'data' in event and isinstance(event['data'], str):
        return event['data'] 
    
    
    # アシスタントからのテキストメッセージを処理
    if 'event' in event:
        event_data = event['event']
        
        # ツール使用の開始
        if 'contentBlockStart' in event_data and 'start' in event_data['contentBlockStart']:
            if 'toolUse' in event_data['contentBlockStart']['start']:
                tool_info = event_data['contentBlockStart']['start']['toolUse']
                return f"\n\n[実行中: {tool_info['name']}]\n\n"        

    return ""

@app.entrypoint
async def strands_agent_bedrock_streaming(payload):
    """
    ストリーミング機能付きでエージェントを呼び出す
    この関数は、非同期ジェネレーターを使用して
    AgentCore Runtime でストリーミングレスポンスを実装する方法を示します
    """
    user_input = payload.get("prompt")
    #print("ユーザー入力:", user_input)
    
    try:
        # 各チャンクを利用可能になり次第ストリーム
        async for event in agent.stream_async(user_input):
            text = parse_event(event)
            if text:  # 空でないレスポンスのみ返す
                yield text
                
            #if "data" in event:
            #    yield event["data"]
            
    except Exception as e:
        # ストリーミングコンテキストでエラーを適切に処理
        error_response = {"error": str(e), "type": "stream_error"}
        print(f"ストリーミングエラー: {error_response}")
        yield error_response


if __name__ == "__main__":
    app.run()

#### エージェントの起動：

同様に、configure_runtime ヘルパー関数を使用して、エージェントデプロイメントに必要な .bedrock_agentcore.yaml、.dockerignore、Dockerfile を作成します。その後、オーケストレーターエージェントランタイムで .launch() を呼び出して、イメージを ECR にプッシュし、AWS 環境に AgentCore Runtime を作成します。

In [ ]:
_, orchestrator_agentcore_runtime = configure_runtime("orchestrator_agent", orchestrator_iam_role, "orchestrator_agent.py")
orchestrator_launch_result = orchestrator_agentcore_runtime.launch()

#### エージェントのテスト

オーケストレーターエージェント AgentCore Runtime のステータスを確認し、使用準備ができていることを確認しましょう。

今回は、utils の `invoke_agent_with_boto3` 関数を使用してオーケストレーターエージェントをテストできます。テクニカルサポートエージェントと HR エージェントの両方の呼び出しをトリガーするはずの質問をオーケストレーターに聞いてみましょう。

In [ ]:
status = check_status(orchestrator_agentcore_runtime)
print(f"ステータス: {status}")

from invoke_agent_utils import invoke_agent_with_boto3


result = invoke_agent_with_boto3 (orchestrator_launch_result.agent_arn, "tell me about my benefits, also tell me how to connect a bluetooth mouse to my mac")

## クリーンアップ（オプション）

作成した AgentCore Runtime をクリーンアップしましょう

In [ ]:
print(f"オーケストレーター: {orchestrator_launch_result.ecr_uri}, {orchestrator_launch_result.agent_id}, {orchestrator_launch_result.ecr_uri.split('/')[1]}")
print(f"HR: {hr_launch_result.ecr_uri}, {hr_launch_result.agent_id}, {hr_launch_result.ecr_uri.split('/')[1]}")
print(f"テクニカル: {tech_launch_result.ecr_uri}, {tech_launch_result.agent_id}, {tech_launch_result.ecr_uri.split('/')[1]}")

In [ ]:
def clean_up_agent_runtimes(launch_result):
    agentcore_control_client = boto3.client(
        'bedrock-agentcore-control',
        region_name=os.getenv("AWS_DEFAULT_REGION")
    )
    ecr_client = boto3.client(
        'ecr',
        region_name=os.getenv("AWS_DEFAULT_REGION")
        
    )
    runtime_delete_response = agentcore_control_client.delete_agent_runtime(
        agentRuntimeId=launch_result.agent_id,
    )

    response = ecr_client.delete_repository(
        repositoryName=launch_result.ecr_uri.split('/')[1],
        force=True
    )

    return response

def delete_iam_roles(agentcore_iam_role):
    iam_client = boto3.client('iam')
    policies = iam_client.list_role_policies(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        MaxItems=100
    )

    for policy_name in policies['PolicyNames']:
        iam_client.delete_role_policy(
            RoleName=agentcore_iam_role['Role']['RoleName'],
            PolicyName=policy_name
        )
    iam_response = iam_client.delete_role(
        RoleName=agentcore_iam_role['Role']['RoleName']
    )
    return iam_response

In [ ]:
print(clean_up_agent_runtimes(hr_launch_result))
print(clean_up_agent_runtimes(tech_launch_result))
print(clean_up_agent_runtimes(orchestrator_launch_result))
print(delete_iam_roles(tech_agent_iam_role))
print(delete_iam_roles(hr_agent_iam_role))
print(delete_iam_roles(orchestrator_iam_role))

# おめでとうございます！